# Bitcoin Test Framework Example

This is a Jupyter notebook. We're going to use it to step through an example test using the Bitcoin Test Framework.

To run the code in a cell, press shift-enter.

# Housekeeping

Let's set up some variables first. Normally, these are set by the test framework itself.

We'll also import some useful utility functions.

In [35]:
from notebook_utils import *  # Sorry for the wildcard. It just makes things easier.

srcdir = "../bitcoin/src" # Change this to the path for your bitcoind executable
cachedir = "./cache" # Change this to a path for a cache directory

In [33]:
# First we import everything we need

# Imports should be in PEP8 ordering (std library first, then third party
# libraries then local imports).
from collections import defaultdict
from pprint import pprint

# Avoid wildcard * imports if possible
from test_framework.blocktools import (create_block, create_coinbase)
from test_framework.mininode import (
    CInv,
    NetworkThread,
    NodeConn,
    NodeConnCB,
    mininode_lock,
    msg_block,
    msg_getdata,
)
from test_framework.test_framework import BitcoinTestFramework
from test_framework.util import (
    assert_equal,
    connect_nodes,
    p2p_port,
    wait_until,
)

# NodeConnCB

`NodeConnCB` is a class containing callbacks to be executed when a P2P message is received from the node-under-test. Subclass `NodeConnCB` and override the `on_*()` methods if you need custom behaviour.

In [3]:
class BaseNode(NodeConnCB):
    def __init__(self):
        """Initialize the NodeConnCB

        Used to inialize custom properties for the Node that aren't
        included by default in the base class. Be aware that the NodeConnCB
        base class already stores a counter for each P2P message type and the
        last received message of each type, which should be sufficient for the
        needs of most tests.

        Call super().__init__() first for standard initialization and then
        initialize custom properties."""
        super().__init__()
        # Stores a dictionary of all blocks received
        self.block_receive_map = defaultdict(int)

    def on_block(self, conn, message):
        """Override the standard on_block callback

        Store the hash of a received block in the dictionary."""
        message.block.calc_sha256()
        self.block_receive_map[message.block.sha256] += 1

    def on_inv(self, conn, message):
        """Override the standard on_inv callback"""
        pass


# Custom functionality

We can define our own helper functions in a test.

In [4]:
def custom_function():
    """Do some custom behaviour

    If this function is more generally useful for other tests, consider
    moving it to a module in test_framework."""
    # self.log.info("running custom_function")  # Oops! Can't run self.log outside the BitcoinTestFramework
    pass

# Setting up the test

Each functional test is a subclass of the BitcoinTestFramework class. Override the `set_test_params()`, `add_options()`, `setup_chain()`, `setup_network()` and `setup_nodes()` methods to customize the test setup as required.

In [5]:
class ExampleTest(BitcoinTestFramework):
    def set_test_params(self):
        """Override test parameters for your individual test.

        This method must be overridden and num_nodes must be exlicitly set."""
        self.setup_clean_chain = True
        self.num_nodes = 3
        # Use self.extra_args to change command-line arguments for the nodes
        self.extra_args = [[], ["-logips"], []]

        # self.log.info("I've finished set_test_params")  # Oops! Can't run self.log before run_test()

    # Use add_options() to add specific command-line options for your test.
    # In practice this is not used very much, since the tests are mostly written
    # to be run in automated environments without command-line options.
    # def add_options()
    #     pass

    # Use setup_chain() to customize the node data directories. In practice
    # this is not used very much since the default behaviour is almost always
    # fine
    # def setup_chain():
    #     pass

    def setup_network(self):
        """Setup the test network topology

        Often you won't need to override this, since the standard network topology
        (linear: node0 <-> node1 <-> node2 <-> ...) is fine for most tests.

        If you do override this method, remember to start the nodes, assign
        them to self.nodes, connect them and then sync."""

        self.setup_nodes()

        # In this test, we're not connecting node2 to node0 or node1. Calls to
        # sync_all() should not include node2, since we're not expecting it to
        # sync.
        connect_nodes(self.nodes[0], 1)
        self.sync_all([self.nodes[0:1]])

    # Use setup_nodes() to customize the node start behaviour (for example if
    # you don't want to start all nodes at the start of the test).
    # def setup_nodes():
    #     pass

    def custom_method(self):
        """Do some custom behaviour for this test

        Define it in a method here because you're going to use it repeatedly.
        If you think it's useful in general, consider moving it to the base
        BitcoinTestFramework class so other tests can use it."""

        self.log.info("Running custom_method")

# Defining the actual test

For a normal test, we'd override the `run_test()` method to define the steps in the test.

Since this is a step-by-step walkthrough, we'll just run the commands manually.

In [6]:
self = ExampleTest()  # Gross. I'm really sorry
setup_test(self, srcdir, cachedir)  # This gets us to the start of the test

2017-11-03 22:57:06.265000 TestFramework (INFO): Initializing test directory /tmp/user/1000/testy5fyqx7b


# Now, let's have some fun

Ok, we're good to go. Our `setup_network()` method has started three nodes for us and set up a P2P connection between node0 and node1. Let's make sure that we really do have three nodes ready for testing:

In [23]:
# First make sure that self.nodes has three objects
print(self.nodes)

[<test_framework.test_node.TestNode object at 0x7f511c144da0>, <test_framework.test_node.TestNode object at 0x7f511c144cc0>, <test_framework.test_node.TestNode object at 0x7f511c144ef0>]


In [31]:
# Each of those objects is a `TestNode` class. We can make calls on that class, which get mapped through to RPC calls
# Let's get some info from those nodes:

print("Node0's blockchain info:\n")
pprint(self.nodes[0].getblockchaininfo())

Node0's blockchain info:

{'bestblockhash': '0f9188f13cb7b2c71f2a335e3a4fc328bf5beb436012afca590b1a11466e2206',
 'bip9_softforks': {'csv': {'since': 0,
                            'startTime': 0,
                            'status': 'defined',
                            'timeout': 999999999999},
                    'segwit': {'since': 0,
                               'startTime': 0,
                               'status': 'defined',
                               'timeout': 999999999999}},
 'blocks': 0,
 'chain': 'regtest',
 'chainwork': '0000000000000000000000000000000000000000000000000000000000000002',
 'difficulty': Decimal('4.656542373906925E-10'),
 'headers': 0,
 'mediantime': 1296688602,
 'pruned': False,
 'size_on_disk': 293,
 'softforks': [{'id': 'bip34', 'reject': {'status': False}, 'version': 2},
               {'id': 'bip66', 'reject': {'status': False}, 'version': 3},
               {'id': 'bip65', 'reject': {'status': False}, 'version': 4}],
 'verificationprogress': 1,

In [30]:
print("Node1's peer info:\n")
pprint(self.nodes[1].getpeerinfo())

Node1's peer info:

[{'addnode': False,
  'addr': '127.0.0.1:49052',
  'addrbind': '127.0.0.1:14045',
  'banscore': 0,
  'bytesrecv': 977,
  'bytesrecv_per_msg': {'feefilter': 32,
                        'getaddr': 24,
                        'getheaders': 93,
                        'ping': 288,
                        'pong': 288,
                        'sendcmpct': 66,
                        'sendheaders': 24,
                        'verack': 24,
                        'version': 138},
  'bytessent': 953,
  'bytessent_per_msg': {'feefilter': 32,
                        'getheaders': 93,
                        'ping': 288,
                        'pong': 288,
                        'sendcmpct': 66,
                        'sendheaders': 24,
                        'verack': 24,
                        'version': 138},
  'conntime': 1509749826,
  'id': 0,
  'inbound': True,
  'inflight': [],
  'lastrecv': 1509750786,
  'lastsend': 1509750786,
  'minping': Decimal('0.000168'),
  

In [32]:
print("Node2's network info:\n")
pprint(self.nodes[2].getnetworkinfo())

Node2's network info:

{'connections': 0,
 'incrementalfee': Decimal('0.00001000'),
 'localaddresses': [],
 'localrelay': True,
 'localservices': '000000000000000d',
 'networkactive': True,
 'networks': [{'limited': False,
               'name': 'ipv4',
               'proxy': '',
               'proxy_randomize_credentials': False,
               'reachable': True},
              {'limited': False,
               'name': 'ipv6',
               'proxy': '',
               'proxy_randomize_credentials': False,
               'reachable': True},
              {'limited': True,
               'name': 'onion',
               'proxy': '',
               'proxy_randomize_credentials': False,
               'reachable': False}],
 'protocolversion': 70015,
 'relayfee': Decimal('0.00001000'),
 'subversion': '/Satoshi:0.15.99(testnode2)/',
 'timeoffset': 0,
 'version': 159900,
 'warnings': 'This is a pre-release test build - use at your own risk - do not '
             'use for mining or merchan

# Shutdown

Now that we've finished, call `shutdown_test()` to shutdown the nodes and clean up resources.